In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

#Original_Data
file_path = '/content/sample_data/OnlineRetail.csv'
data = pd.read_csv(file_path, encoding= 'unicode_escape')
Data_copy =data
print(data.head())

  InvoiceNo StockCode                     Description  Quantity   InvoiceDate  \
0    536369     21756        BATH BUILDING BLOCK WORD         3  12/1/10 8:35   
1    536368     22960        JAM MAKING SET WITH JARS         6  12/1/10 8:34   
2    536370     21883                STARS GIFT TAPE         24  12/1/10 8:45   
3    536370     21913  VINTAGE SEASIDE JIGSAW PUZZLES        12  12/1/10 8:45   
4    536367     48187             DOORMAT NEW ENGLAND         4  12/1/10 8:34   

   UnitPrice  CustomerID         Country  
0       5.95     13047.0  United Kingdom  
1       4.25     13047.0  United Kingdom  
2       0.65     12583.0          France  
3       3.75     12583.0          France  
4       7.95     13047.0  United Kingdom  


In [ ]:
import pandas as pd
import numpy as np

# Handling NA values in the CustomerID field
na_count = data['CustomerID'].isna().sum()
print(f'{na_count} NA in CustomerID')

# Let's omit NA values
data = data.dropna(subset=['CustomerID'])

# Aggregate the data if needed (e.g., by summing 'Quantity' for each unique combination of 'CustomerID' and 'StockCode')
data = data.groupby(['CustomerID', 'StockCode'])['Quantity'].sum().reset_index()

# Building a customer-item matrix
CustomerItemMatrix = data.pivot(index='CustomerID', columns='StockCode', values='Quantity')

# Fill NaN values with 0 (no purchase)
CustomerItemMatrix = CustomerItemMatrix.fillna(0)

# Let's encode 0-1 where 1 means that the product was purchased (at least once)
# and 0 means that the product was never purchased by the given customer
CustomerItemMatrix = CustomerItemMatrix.apply(lambda x: (x > 0).astype(int))


135080 NA in CustomerID


In [ ]:
CustomerItemMatrix.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214Y,90214Z,BANK CHARGES,C2,CRUK,D,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


# Extract the relevant columns (exclude the 'CustomerID' column)
user_features = CustomerItemMatrix.iloc[:, 1:]

# Calculate cosine similarities between users
UserToUser = cosine_similarity(user_features.T)

# Create a DataFrame with default integer indices (no need to specify columns and index)
UserToUser_df = pd.DataFrame(UserToUser)

# Print the first few rows of the similarity matrix
print(UserToUser_df.head())



   0        1        2         3         4         5         6         7     \
0   1.0  0.00000  0.00000  0.000000  0.000000  0.032774  0.045883  0.047836   
1   0.0  1.00000  0.11547  0.000000  0.000000  0.057143  0.060000  0.041703   
2   0.0  0.11547  1.00000  0.000000  0.000000  0.164957  0.000000  0.000000   
3   0.0  0.00000  0.00000  1.000000  0.447214  0.063888  0.044721  0.000000   
4   0.0  0.00000  0.00000  0.447214  1.000000  0.071429  0.050000  0.000000   

       8         9     ...  3673  3674  3675  3676  3677  3678  3679  \
0  0.000000  0.000000  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1  0.060302  0.060302  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2  0.000000  0.000000  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3  0.000000  0.000000  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4  0.000000  0.000000  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

       3680  3681  3682  
0  0.017244   0.0   0.0  
1  0.075165   0.0   0.0  
2  0.000000   

In [ ]:
CustomerItemMatrix.index

Float64Index([12346.0, 12347.0, 12348.0, 12349.0, 12350.0, 12352.0, 12353.0,
              12354.0, 12355.0, 12356.0,
              ...
              18273.0, 18274.0, 18276.0, 18277.0, 18278.0, 18280.0, 18281.0,
              18282.0, 18283.0, 18287.0],
             dtype='float64', name='CustomerID', length=4372)

In [ ]:
import pandas as pd

# Similarity matrix UserToUser is already calculated

# Check if the customer with ID '12348.0' exists in your data
customer_id = 12348.0
if customer_id in CustomerItemMatrix.index:
    # Rank the most similar customers to our customer with ID 12350
    top_10_similar = CustomerItemMatrix.index[
        UserToUser[CustomerItemMatrix.index.get_loc(customer_id)].argsort()[::-1][:11]
    ]

    # Find what the customer A (12348.0) bought
    bought_by_A = CustomerItemMatrix.loc[customer_id]
    bought_by_A = bought_by_A[bought_by_A != 0].index.tolist()
else:
    print(f"Customer with ID {customer_id} not found in the dataset.")
    top_10_similar = []
    bought_by_A = []


# Find what the B customer bought
customer_B_id = 18287.0
if customer_B_id in top_10_similar:
    bought_by_B = CustomerItemMatrix.loc[customer_B_id]
    bought_by_B = bought_by_B[bought_by_B != 0].index.tolist()
else:
    print(f"Customer with ID {customer_B_id} not found in the top 10 similar customers.")
    bought_by_B = []

# Find the items that customer B didn't buy
recommend_to_B = set(bought_by_A) - set(bought_by_B)

# Find the descriptions of these items from the 'data' DataFrame
recommend_to_B_description = Data_copy[Data_copy['StockCode'].isin(recommend_to_B)][['StockCode', 'Description']].drop_duplicates()
recommend_to_B_description = recommend_to_B_description[recommend_to_B_description['StockCode'].isin(recommend_to_B)]

print("\n Print the list of item descriptions as a recommendation to B\n")
print(recommend_to_B_description)


Customer with ID 18287.0 not found in the top 10 similar customers.

 Print the list of item descriptions as a recommendation to B

       StockCode                         Description
28          POST                             POSTAGE
98         21977  PACK OF 60 PINK PAISLEY CAKE CASES
99         84991         60 TEATIME FAIRY CAKE CASES
237        21983    PACK OF 12 BLUE PAISLEY TISSUES 
238        21980   PACK OF 12 RED RETROSPOT TISSUES 
406        21213         PACK OF 72 SKULL CAKE CASES
409        84992      72 SWEETHEART FAIRY CAKE CASES
623        22952     60 CAKE CASES VINTAGE CHRISTMAS
649        22616          PACK OF 12 LONDON TISSUES 
659        22951     60 CAKE CASES DOLLY GIRL DESIGN
796        21985   PACK OF 12 HEARTS DESIGN TISSUES 
1263       21982            PACK OF 12 SUKI TISSUES 
1264       21981        PACK OF 12 WOODLAND TISSUES 
1265       21967            PACK OF 12 SKULL TISSUES
1579       22437       SET OF 9 BLACK SKULL BALLOONS
1834       84988  SE